<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Mini%20Datathon/Notebooks/Quix_Streams_PROCESS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Quix Streams
Just use pip install to download the Quix Streams library. 

[Quix Streams](https://github.com/quixio/quix-streams) is an open source Python library for processing streaming data. It’s aimed at people who work with time-series data streams — from developers and ML engineers to data scientists and data engineers.

In [2]:
! pip install quixstreams

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import the libraries
We will be using mainly pandas, quix, matplotlib and seaborn.

In [3]:
import pandas as pd
import quixstreams as qx

# 1 - Create client
Let's start by creating a Quix client that we'll use to publish and subscribe to Kafka topics.

In [4]:
token = 'sdk-296f2b9decff4770a525ff7d8855a78d'
client = qx.QuixStreamingClient(token)
client

# 2 - Producer and consumer  clients
To suscribe to data from one topic, we will need to create a consumer client pointing to that topic.

In [5]:
topic_name = "test-topic"
topic_consumer = client.get_topic_consumer(topic_name)
topic_consumer

In [6]:
topic_name = "test-topic-II"
topic_producer = client.get_topic_producer(topic_name)
topic_producer

# 3 - Processing data
Consume data from a topic, process it and publish it to another.

In [8]:
def on_stream_received_handler(stream_received: qx.StreamConsumer):
  stream_received.timeseries.on_dataframe_received = on_timeseries_data_received_handler

def on_timeseries_data_received_handler(stream_in: qx.StreamConsumer, df: pd.DataFrame):
  # Process data
  print("Data from stream " + stream_in.stream_id)
  display(df)
  df["Param A"] = 1000 * df["Param A"]
  display(df)

  # Stream
  stream_out = topic_producer.get_or_create_stream("your-name")
  stream_out.timeseries.publish(df)

topic_producer = client.get_topic_producer("test-topic-II")
topic_consumer = client.get_topic_consumer("test-topic")
topic_consumer.on_stream_received = on_stream_received_handler
qx.App.run()

Data from stream test-stream_1


,timestamp,Param A,Column B
0,1681557517290106000,10.0,12.0
1,1681557522290684000,20.0,9.0


,timestamp,Param A,Column B
0,1681557517290106000,10000.0,12.0
1,1681557522290684000,20000.0,9.0


Data from stream test-stream_2


,timestamp,Param A,Column B
0,1681557517290106000,10.0,12.0
1,1681557522290684000,20.0,9.0


,timestamp,Param A,Column B
0,1681557517290106000,10000.0,12.0
1,1681557522290684000,20000.0,9.0


In [ ]:
df